In [ ]:
%%time
#create one-hot-encoding dataframe for category_2 and category_3
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

#reduce memory usage
historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)
df_merchant=reduce_mem_usage(df_merchant)



In [ ]:
df_merchant['abs_numerical_1'] = abs(df_merchant['numerical_1'])+1
cols = ['merchant_id','abs_numerical_1']
df_trick = df_merchant[cols]
df_trick.head()


#     historical_transactions['today_date'] = pd.to_datetime('2018-2-1')
#     historical_transactions['day_diff'] = (historical_transactions['today_date'] - historical_transactions['purchase_date']).dt.days
#     historical_transactions = historical_transactions(right = df_trick , how='left', on = 'merchant_id')
#     historical_transactions['2w_purchase'] = np.where(historical_transactions['day_diff'] <= 14,historical_transactions['purchase_amount'],0)
#     historical_transactions['1m_purchase'] = np.where(historical_transactions['day_diff'] <= 30,historical_transactions['purchase_amount'],0)
#     historical_transactions['3m_purchase'] = np.where(historical_transactions['day_diff'] <= 90,historical_transactions['purchase_amount'],0)
#     historical_transactions['6m_purchase'] = np.where(historical_transactions['day_diff'] <= 180,historical_transactions['purchase_amount'],0)
#     historical_transactions['1y_purchase'] = np.where(historical_transactions['day_diff'] <= 365,historical_transactions['purchase_amount'],0)

In [ ]:
historical_transactions.head()

In [ ]:

#Amount of purchase in the last month
# for df in [historical_transactions,new_transactions]:
def purchase_lapse(df):
    df['today_date'] = pd.to_datetime('2018-2-1')
    df['day_diff'] = (df['today_date'] - df['purchase_date']).dt.days
    df = df.merge(right = df_trick , how='left', on = 'merchant_id')
    df['purchase_amount'] = df['purchase_amount']*df['abs_numerical_1']
    df['2w_purchase'] = np.where(df['day_diff'] <= 14,df['purchase_amount'],0)
    df['1m_purchase'] = np.where(df['day_diff'] <= 30,df['purchase_amount'],0)
    df['3m_purchase'] = np.where(df['day_diff'] <= 90,df['purchase_amount'],0)
    df['6m_purchase'] = np.where(df['day_diff'] <= 180,df['purchase_amount'],0)
    df['1y_purchase'] = np.where(df['day_diff'] <= 365,df['purchase_amount'],0)
    return df

historical_transactions = purchase_lapse(historical_transactions)
new_transactions = purchase_lapse(new_transactions)


In [ ]:
agg_fun = {'authorized_flag': ['sum', 'mean']}

#groupby ohe dataframe by card_id
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]
gc.collect()

In [ ]:
%%time
#create purchase month
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month
gc.collect()

In [ ]:
%%time
def aggregate_transactions(history):
    '''
    Parameters 
    -----------
    history: historical transaction of dataframe
    
    
    output:
    ----------
    dataframe groupby card_id and aggregated on multiple function
    '''
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    
    agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        '2w_purchase':['sum','mean','max','min','std'],
        '1m_purchase':['sum','mean','max','min','std'],
        '3m_purchase':['sum','mean','max','min','std'],
        '6m_purchase':['sum','mean','max','min','std'],
        '1y_purchase':['sum','mean','max','min','std'],
#         'merchant_category_id': ['nunique'],
#         'state_id': ['nunique'],
#         'city_id': ['nunique'],
#         'subsector_id': ['nunique'],
        'purchase_amount': ['sum',
                            'mean',
                            'max',
                            'min',
                            'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['min', 'max','std','sum',]
        }
    #groupby card id 
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    #merge the two dataframe together 
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

gc.collect()